In [1]:
import nltk
import konlpy
import numpy as np
import pandas as pd

C:\Users\thinp\anaconda3\envs\Wyatt37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# 01. Tokenization

자연어 처리에서 크롤링 등으로 얻어낸 코퍼스(말뭉치) 데이터를 전처리하는 것이 아주 핵심이다. 종류로는

- 토큰화
- 정제
- 정규화

가 있다. 코퍼스를 토큰 단위로 나누는 작업을 토큰화라고 한다. 주로 NLTK와 KoNLPY를 활용한다.

## 01.1. Word Tokenization

### 영어의 어퍼스트로피가 들어있는 단어는 어떻게 토큰화할까?

In [3]:
corpus = "Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."

In [2]:
from nltk.tokenize import word_tokenize

In [4]:
print(word_tokenize(corpus))

['Do', "n't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr.', 'Jone', "'s", 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


word_tokenize는 Don't를 Do와 n't로 분리했습니다. Jone's는 Jone 과 's로 분리했습니다.

In [5]:
from nltk.tokenize import WordPunctTokenizer

In [6]:
print(WordPunctTokenizer().tokenize(corpus))

['Don', "'", 't', 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr', '.', 'Jone', "'", 's', 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


WordPunctTokenizer는 Don, ', t 로 분리 했습니다.

In [7]:
from keras.preprocessing.text import text_to_word_sequence

In [8]:
print(text_to_word_sequence(corpus))

["don't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', 'mr', "jone's", 'orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']


케라스의 text_to_word_sequence는 온점, 컴마, 느낌표도 전부 제거하며, 어퍼스트로피를 보존합니다.

### 토큰화는 무조건 없애는 게 좋을까?

1) 구두점이나 특수 문자를 단순 제외해서는 안 된다.

ph.d 라든지 AT&T 라든지 01/02/06 이라든지 45.55 라든지 아무튼 세심하게 고민해야 함

2) 줄임말과 단어 내에 띄어쓰기가 있는 경우

New York 이라든지, rock 'n' roll 이라든지

### 표준 토큰화 예제

표준으로 쓰고 있는 토큰화 방법 중 하나인 Penn Treebank Tokenization 규칙을 보도록 하겠습니다.

- 규칙1. 하이픈으로 구성된 단어는 하나로 유지한다.
- 규직2. doesn't 와 같이 어퍼스트로피로 '접어'가 함께하는 단어는 분리해준다.

In [9]:
corpus = "Starting a home-based restaurant may be an ideal. it doesn't have a food chain or restaurant of their own."

In [10]:
from nltk.tokenize import TreebankWordTokenizer

In [11]:
tokenizer = TreebankWordTokenizer()

In [12]:
print(tokenizer.tokenize(corpus))

['Starting', 'a', 'home-based', 'restaurant', 'may', 'be', 'an', 'ideal.', 'it', 'does', "n't", 'have', 'a', 'food', 'chain', 'or', 'restaurant', 'of', 'their', 'own', '.']


home-based 를 하나로 보며, does와 n't는 분리한 것을 알 수 있습니다. 온점도 유지 되었구요.

## 01.2. Sentence Tokenization

문장 구분이 되어있지 않은 말뭉치라면 문장 구분을 해주어야 합니다. 다만 쉽지가 않습니다. 기준이 애매하기 때문이죠.

NLTK의 sent_tokenize 를 살펴보겠습니다.

In [13]:
from nltk.tokenize import sent_tokenize

In [14]:
corpus = "His barber kept his word. But keeping such a huge secret to himself was driving him crazy. Finally, the barber went up a mountain and almost to the edge of a cliff. He dug a hole in the midst of some reeds. He looked about, to mae sure no one was near."

In [15]:
print(sent_tokenize(corpus))

['His barber kept his word.', 'But keeping such a huge secret to himself was driving him crazy.', 'Finally, the barber went up a mountain and almost to the edge of a cliff.', 'He dug a hole in the midst of some reeds.', 'He looked about, to mae sure no one was near.']


잘 했습니다. 그렇다면 온점이 많은 문단도 가능 할까요?!

In [16]:
corpus = "I am actively looking for Ph.D. students. and you are a Ph.D student."

In [17]:
print(sent_tokenize(corpus))

['I am actively looking for Ph.D. students.', 'and you are a Ph.D student.']


NLTK는 단순히 온점을 구분자로 문장을 구분하지 않기 때문에 잘 분리합니다.

한국어도 되는지 살펴봅시다.

In [19]:
from konlpy.tag import Kkma
from konlpy.utils import pprint

In [ ]:
kkma = Kkma()

In [21]:
pprint(kkma.sentences(u'네, 안녕하세요. 반갑습니다.'))

['네, 안녕하세요.', '반갑습니다.']


In [22]:
pprint(kkma.nouns(u'질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

['질문', '건의', '건의사항', '사항', '깃헙', '이슈', '트래커']


In [23]:
pprint(kkma.pos(u'오류보고는 실행환경, 에러메세지와함께 설명을 최대한상세히!^^'))

[('오류', 'NNG'),
 ('보고', 'NNG'),
 ('는', 'JX'),
 ('실행', 'NNG'),
 ('환경', 'NNG'),
 (',', 'SP'),
 ('에러', 'NNG'),
 ('메세지', 'NNG'),
 ('와', 'JKM'),
 ('함께', 'MAG'),
 ('설명', 'NNG'),
 ('을', 'JKO'),
 ('최대한', 'NNG'),
 ('상세히', 'MAG'),
 ('!', 'SF'),
 ('^^', 'EMO')]


세상엔 참 천재들이 많습니다.

그 외에 문장 토큰화 오픈 소스로는 NLTK, OpenNLP, 스탠포드 CoreNLP, splitta, LingPipe 등이 있습니다.

한국어는 토큰화가 매우 어렵습니다. 그 이유는 1) 교착어 2) 띄어쓰기 때문입니다.

## 01.3. Part-of-speech tagging

품사 태깅이라고 합니다. fly는 날다 와 파리 라는 뜻을 가지고 있습니다.그래서 품사 태깅이 중요하죠.

### NLTK와 KoNLPY를 이용한 영어, 한국어 토큰화 실습

In [24]:
text="I am actively looking for Ph.D. students. and you are a Ph.D. student."
print(word_tokenize(text))

['I', 'am', 'actively', 'looking', 'for', 'Ph.D.', 'students', '.', 'and', 'you', 'are', 'a', 'Ph.D.', 'student', '.']


In [25]:
from nltk.tag import pos_tag

In [26]:
x = word_tokenize(text)
pos_tag(x)

[('I', 'PRP'),
 ('am', 'VBP'),
 ('actively', 'RB'),
 ('looking', 'VBG'),
 ('for', 'IN'),
 ('Ph.D.', 'NNP'),
 ('students', 'NNS'),
 ('.', '.'),
 ('and', 'CC'),
 ('you', 'PRP'),
 ('are', 'VBP'),
 ('a', 'DT'),
 ('Ph.D.', 'NNP'),
 ('student', 'NN'),
 ('.', '.')]

한국어 자연어 처리를 위해서는 KoNLPy를 사용합니다.

- Okt(Open Korea Text)
- Mecab(메캅)
- Komoran(코모란)
- Hannanum(한나눔)
- Kkma(꼬꼬마)

가 있습니다.

In [29]:
corpus ="열심히 코딩한 당신, 연휴에는 여행을 가봐요"

In [27]:
from konlpy.tag import Okt

In [28]:
okt=Okt()

1. morphs: 형태소 추출
2. pos: 품사 태깅
3. nouns: 명사 추출

In [30]:
print(okt.morphs(corpus))

['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']


In [31]:
print(okt.pos(corpus))

[('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]


In [32]:
print(okt.nouns(corpus))

['코딩', '당신', '연휴', '여행']


In [33]:
print(kkma.morphs(corpus))

['열심히', '코딩', '하', 'ㄴ', '당신', ',', '연휴', '에', '는', '여행', '을', '가보', '아요']


In [34]:
print(kkma.pos(corpus))

[('열심히', 'MAG'), ('코딩', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('당신', 'NP'), (',', 'SP'), ('연휴', 'NNG'), ('에', 'JKM'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('가보', 'VV'), ('아요', 'EFN')]


In [35]:
print(kkma.nouns(corpus))

['코딩', '당신', '연휴', '여행']


결과가 다소 다릅니다.

한국어 형태소 분석기 성능 비교 : https://iostream.tistory.com/144
http://www.engear.net/wp/%ED%95%9C%EA%B8%80-%ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EA%B8%B0-%EB%B9%84%EA%B5%90/

위 링크를 정리하면,

- Okt 는 속도는 빠르나 품질이 좋지 않다.
- Kkma 는 품질은 좋으나 속도가 매우 느리다.
- mecab은 속도도 매우 빠르고 품질도 가장 좋다. 다만 윈도우에서 활용이 어렵다.

# 02. Cleaning and Normalization

- 정제(cleaning) : 갖고 있는 코퍼스로부터 노이즈 데이터를 제거한다.
- 정규화(normalization) : 표현 방법이 다른 단어들을 통합시켜서 같은 단어로 만들어준다.

## 02.1 규칙에 기반한 표기가 다른 언어들의 통합

USA와 US는 같습니다. uh-huh와 uhhuh는 같습니다.

## 02.2. 대소문자 통합

대부분의 글은 소문자로 작성되기 때문에 소문자로 변환합니다.

Automobile과 automobile은 같습니다. 다만 US와 us는 다릅니다. 또 회사 이름이나 사람 이름은 대문자로 유지되는 게 맞습니다.

## 02.3. 불필요한 단어의 제거

# 03. 어간 추출(Stemming) and 표제어 추출(Lemmatization)

## 03.1. 표제어 추출

정규화 기법 중 코퍼스에 있는 단어의 개수를 줄일 수 있는 표제어 추출과 어간 추출을 알아보겠습니다.

표제어란 뿌리 단어를 찾는 것입니다. am, are, is 는 전부 be로부터 파생되었습니다. 그래서 이 친구들의 표제어는 be 입니다.

표제어 추출을 하는 가장 섬세한 방법은 형태학적 파싱을 먼저 진행하는 것입니다. 형태소는 두 가지 종류가 있습니다.

- 어간(stem) : 단어의 의미를 담고 있는 단어의 핵심 부분
- 접사(affix) : 단어에 추가적인 의미를 주는 부분

NLTK에서는 표제어 추출을 위한 WordNetLemmatizer를 지원합니다.

In [36]:
from nltk.stem import WordNetLemmatizer

In [60]:
n = WordNetLemmatizer()

In [61]:
words=['policy', 'doing', 'organization', 'have',
       'going', 'love', 'lives', 'fly',
       'dies', 'watched', 'has', 'starting']

In [63]:
print([n.lemmatize(w) for w in words])

['policy', 'doing', 'organization', 'have', 'going', 'love', 'life', 'fly', 'dy', 'watched', 'ha', 'starting']


이상하시죠? dy, ha 라고 추출을 하니까요. lemmatizer는 품사 정보를 알아야만 정확한 결과를 알려줍니다.

In [64]:
n.lemmatize('dies', 'v')

'die'

In [66]:
n.lemmatize('watched', 'v')

'watch'

In [68]:
n.lemmatize('has', 'v')

'have'

## 03.2. 어간 추출

어간 추출은 형태학적 분석을 단순화한 버전이라고 볼 수도 있고, 정해진 규칙만 보고 단어의 어미를 자르는 어림짐작의 작업이라고 볼 수도 있습니다.

예제를 통해 포터 알고리즘을 살펴봅시다.

In [69]:
corpus = "This was not the map we found in Billy Bones's chest, but an accurate copy, complete in all things--names and heights and soundings--with the single exception of the red crosses and the written notes."

In [70]:
from nltk.stem import PorterStemmer

In [71]:
s = PorterStemmer()

In [79]:
words = word_tokenize(corpus)

In [73]:
print(words)

['This', 'was', 'not', 'the', 'map', 'we', 'found', 'in', 'Billy', 'Bones', "'s", 'chest', ',', 'but', 'an', 'accurate', 'copy', ',', 'complete', 'in', 'all', 'things', '--', 'names', 'and', 'heights', 'and', 'soundings', '--', 'with', 'the', 'single', 'exception', 'of', 'the', 'red', 'crosses', 'and', 'the', 'written', 'notes', '.']


In [74]:
print([s.stem(w) for w in words])

['thi', 'wa', 'not', 'the', 'map', 'we', 'found', 'in', 'billi', 'bone', "'s", 'chest', ',', 'but', 'an', 'accur', 'copi', ',', 'complet', 'in', 'all', 'thing', '--', 'name', 'and', 'height', 'and', 'sound', '--', 'with', 'the', 'singl', 'except', 'of', 'the', 'red', 'cross', 'and', 'the', 'written', 'note', '.']


가령, 포터 알고리즘의 어간 추출은 이러한 규칙들을 가집니다.
- ALIZE → AL
- ANCE → 제거
- ICAL → IC

In [75]:
words=['formalize', 'allowance', 'electricical']
print([s.stem(w) for w in words])

['formal', 'allow', 'electric']


포터 외에 랭커스터 스태머를 비교해볼까요?

In [76]:
from nltk.stem import LancasterStemmer
l=LancasterStemmer()

In [80]:
print([l.stem(w) for w in words])

['thi', 'was', 'not', 'the', 'map', 'we', 'found', 'in', 'bil', 'bon', "'s", 'chest', ',', 'but', 'an', 'acc', 'cop', ',', 'complet', 'in', 'al', 'thing', '--', 'nam', 'and', 'height', 'and', 'sound', '--', 'with', 'the', 'singl', 'exceiv', 'of', 'the', 'red', 'cross', 'and', 'the', 'writ', 'not', '.']


# 04. Stopwords(불용어)

큰 의미가 없는 단어 토큰을 제거하는 작업이 필요합니다. i, my, me, over, 조사 등등 이러한 것들을 stopword라고 합니다.

In [81]:
from nltk.corpus import stopwords

In [82]:
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

위와 같이 nltk에서는 기본적인 불용어 사전을 지원합니다.

In [83]:
example = "Family is not an important thing. It's everything."
stop_words = set(stopwords.words('english')) 

word_tokens = word_tokenize(example)

result = []
for w in word_tokens: 
    if w not in stop_words: 
        result.append(w) 

print(word_tokens) 
print(result) 

['Family', 'is', 'not', 'an', 'important', 'thing', '.', 'It', "'s", 'everything', '.']
['Family', 'important', 'thing', '.', 'It', "'s", 'everything', '.']


In [84]:
example = "고기를 아무렇게나 구우려고 하면 안 돼. 고기라고 다 같은 게 아니거든. 예컨대 삼겹살을 구울 때는 중요한 게 있지."
stop_words = "아무거나 아무렇게나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 하면 아니거든"

stop_words=stop_words.split(' ')
word_tokens = word_tokenize(example)

result = [] 
for w in word_tokens: 
    if w not in stop_words: 
        result.append(w) 
# 위의 4줄은 아래의 한 줄로 대체 가능
# result=[word for word in word_tokens if not word in stop_words]

print(word_tokens) 
print(result)

['고기를', '아무렇게나', '구우려고', '하면', '안', '돼', '.', '고기라고', '다', '같은', '게', '아니거든', '.', '예컨대', '삼겹살을', '구울', '때는', '중요한', '게', '있지', '.']
['고기를', '구우려고', '안', '돼', '.', '고기라고', '다', '같은', '게', '.', '삼겹살을', '구울', '때는', '중요한', '게', '있지', '.']


불용어 사전은 직접 txt파일이나 csv파일로 정리해놓고 불러와서 쓰는 게 좋습니다.